# Boss Zhipin Automation Tutorial

This comprehensive tutorial shows you how to use the Boss Zhipin automation service to interact with candidates, manage conversations, and extract resume data.

## Table of Contents
1. [Setup & Installation](#setup--installation)
2. [Service Management](#service-management)
3. [Basic API Usage](#basic-api-usage)
4. [Chat Management](#chat-management)
5. [Resume Extraction](#resume-extraction)
6. [Candidate Management](#candidate-management)
7. [Advanced Features](#advanced-features)
8. [Troubleshooting](#troubleshooting)


## Setup & Installation

### Prerequisites
- Python 3.8+
- Chrome browser
- Boss Zhipin account


In [30]:
import requests
import json
from datetime import datetime

# Service configuration
BASE_URL = "http://127.0.0.1:5001"

# Check service status
response = requests.get(f"{BASE_URL}/status", timeout=30)
status = response.json()
print("🔍 Service Status:")
print(json.dumps(status, indent=2, ensure_ascii=False))


🔍 Service Status:
{
  "status": "running",
  "logged_in": false,
  "timestamp": "2025-09-30T14:29:08.018981",
  "new_message_count": 46,
  "new_greet_count": 41
}


```
# Install required packages
%pip install requests playwright fastapi uvicorn
%pip install playwright
!playwright install chromium
```

In [34]:
# Trigger login verification
response = requests.get(f"{BASE_URL}/status", timeout=30)
login_result = response.json()
print("🔐 Login Status:")
print(json.dumps(login_result, indent=2, ensure_ascii=False))


🔐 Login Status:
{
  "status": "running",
  "logged_in": true,
  "timestamp": "2025-09-30T14:29:42.163367",
  "new_message_count": 46,
  "new_greet_count": 41
}


## Chat operations

In [44]:
import random
import json
# Get list of chat candidates
response = requests.get(f"{BASE_URL}/chat/dialogs", params={'limit': 3}, timeout=30).json()
chat_id = random.choice(response)['id']
print(f'Chosen Chat ID: {chat_id}')
print(f"👥 Chat dialogs: ")
for message in response:
    print(f"{message['name']}({message['job_title']})|{message['timestamp']}: \n{message['text']}")
    print('-'*100)

Chosen Chat ID: 571157821-0
👥 Chat dialogs: 
李胜利(大模型算法工程师)|昨天: 
刚刚看您发布了这个职位，我很喜欢，如果您觉得合适，请随时联系我，感谢
----------------------------------------------------------------------------------------------------
杨辰宇(大模型算法工程师)|09月28日: 
你好，我想应聘贵公司的大模型算法工程师，期盼回复，谢谢！
----------------------------------------------------------------------------------------------------
李庆旭(大模型算法工程师)|09月27日: 
占用您百忙之中的一点时间，关注贵公司很久了，和我的职业规划很吻合，想和您聊聊
----------------------------------------------------------------------------------------------------


In [5]:
# Example: Send a message to a candidate
# Replace 'your_chat_id' with an actual chat ID from the candidates list
# chat_id = "your_chat_id_here"  # Replace with actual chat ID
message = "您好，我对您的简历很感兴趣，希望能进一步沟通。"

if chat_id != "your_chat_id_here":
    response = requests.post(f"{BASE_URL}/chat/{chat_id}/send", json={'message': message}, timeout=30)
    send_result = response.json()
    
    print("📤 Send Message Result:")
    print(json.dumps(send_result, indent=2, ensure_ascii=False))
else:
    print("⚠️ Please replace 'your_chat_id_here' with an actual chat ID from the candidates list above.")


📤 Send Message Result:
{
  "success": true,
  "chat_id": "555717660-0",
  "message": "您好，我对您的简历很感兴趣，希望能进一步沟通。",
  "details": "消息发送成功",
  "timestamp": "2025-09-29T14:43:49.437138"
}


### Get chat history 

In [ ]:
# Get chat history for a specific conversation
if chat_id != "your_chat_id_here":
    response = requests.get(f"{BASE_URL}/chat/{chat_id}/messages", timeout=30)
    history = response.json()
    print(f"📜 Chat History: {json.dumps(history, indent=2, ensure_ascii=False)}")
else:
    print("⚠️ Please set a valid chat_id first.")


📜 Chat History: {
  "success": true,
  "chat_id": "555717660-0",
  "messages": [
    {
      "type": "candidate",
      "timestamp": "2025-09-25 16:39:00",
      "message": "刚刚看您发布了这个职位，我很喜欢，如果您觉得合适，请随时联系我，感谢",
      "status": null
    },
    {
      "type": "recruiter",
      "timestamp": "2025-09-26 02:52:00",
      "message": "方便发一份简历过来吗？",
      "status": "已读"
    },
    {
      "type": "system",
      "timestamp": "2025-09-26 02:52:00",
      "message": "简历请求已发送",
      "status": null
    },
    {
      "type": "candidate",
      "timestamp": "2025-09-29 15:34:00",
      "message": "张良简历.pdf\n点击预览附件简历",
      "status": null
    },
    {
      "type": "system",
      "timestamp": "2025-09-29 15:34:00",
      "message": "你撤回了一条消息",
      "status": null
    },
    {
      "type": "system",
      "timestamp": "2025-09-29 05:43:00",
      "message": "你撤回了一条消息 重新编辑",
      "status": null
    }
  ],
  "count": 6,
  "timestamp": "2025-09-29T14:50:51.768305"
}


: 

In [11]:
response

<Response [500]>

In [ ]:
# Discard a candidate (mark as not suitable)
print(f"Chat ID: {chat_id}")
if chat_id != "your_chat_id_here":
    response = requests.post(f"{BASE_URL}/candidate/discard", json={'chat_id': chat_id}, timeout=30)
    discard_result = response.json()
    
    print("🗑️ Discard Candidate Result:")
    print(json.dumps(discard_result, indent=2, ensure_ascii=False))
    messages, chat_id = get_chat_candidates()
else:
    print("⚠️ Please set a valid chat_id first.")


Chat ID: 689994060-0
🗑️ Discard Candidate Result:
{
  "success": true,
  "chat_id": "689994060-0",
  "details": "确认已丢弃",
  "timestamp": "2025-09-28T11:46:57.337421"
}
Chat ID: 38187855-0


## Resume operations
### 请求简历
⚠️ needs to be fixed

In [ ]:
# Request resume from a candidate
if chat_id != "your_chat_id_here":
    response = requests.post(f"{BASE_URL}/resume/request", json={'chat_id': chat_id}, timeout=30)
    request_result = response.json()
    
    print("📋 Request Resume Result:")
    print(json.dumps(request_result, indent=2, ensure_ascii=False))
else:
    print("⚠️ Please set a valid chat_id first.")


In [ ]:
# Extract online resume content using advanced capture methods
if chat_id != "your_chat_id_here":
    response = requests.post(f"{BASE_URL}/resume/online", json={'chat_id': chat_id}, timeout=30)
    resume_result = response.json()
    
    print("📄 Resume Extraction Result:")
    if resume_result.get('success'):
        print(f"✅ Success: {resume_result.get('details', 'Resume extracted')}")
        print(f"📊 Capture Method: {resume_result.get('method', 'unknown')}")
        if text :=resume_result.get('text'):
            print(text[:400] + "\n...")
    else:
        print(f"❌ Error: {resume_result.get('error', 'Unknown error')}")
else:
    print("⚠️ Please set a valid chat_id first.")


📄 Resume Extraction Result:
✅ Success: 来自inline简历
📊 Capture Method: unknown
李先生
 刚刚活跃
36岁  9年  硕士  离职-随时到岗
沟通能力强，学习能力强，抗压能力强

期望职位
---
北京自然语言处理算法行业不限30-60K

工作经历
---
研发中心
深度学习/NLP/推荐/运筹
2018.12 - 至今
自然语言处理项目主要负责人
大语言模型项目主要负责人
推荐算法项目主要负责人
运筹优化项目主要负责人

算法人工智能深度学习

教育经历
---
大连理工大学
管理科学与工程
硕士
2013 - 2016

985院校QS世界大学排名TOP500
...


In [ ]:
# 查看候选人离线简历

id = '508516626-0'
response = requests.post(f"{BASE_URL}/resume/view_full", json={'chat_id': chat_id}, timeout=30).json()
if response.get('success'):
    resume = response['content']
    print(len(resume))
    print(resume[0:300])
else:
    print(f"❌ Error: {response.get('error', response.get('details', 'Unknown error'))}")

4065
!"#

☆联系电话：15652968768☆电子邮箱：leeq311@outlook.com

☆通讯地址：北京市朝阳区农展南里社区，100012

$%&'

2013.09-2016.06大连理工大学专业：管理科学与工程全日制统招工学硕士

2008.09-2012.06济宁学院专业：数学与应用数学全日制统招理学学士

()*+

1.蔷薇大树科技有限公司北京分公司高级算法工程师2022.6-至今

主要负责风控业务场景中的算法研究和工程落地，完成相关智能风控系统的技术架构设计、功能研发和迭代维护。

Ø中登网质押资产智能查重系统，通过大数据+人工智能技术实现质押融资担保等业务场景中的质


In [ ]:
response

{'success': False, 'details': '未找到指定对话项'}

## Recommended Candidates Page

### Get Recommended Candidates


In [36]:
# Get recommended candidates from Boss Zhipin
recommended = requests.get(f'{BASE_URL}/recommend/candidates', params={'limit': 5}).json()

print("⭐ Recommended Candidates:")
if recommended.get('success'):
    for i, candidate in enumerate(recommended.get('candidates', [])):
        print(candidate)
        print('-'*100)
else:
    print(f"❌ Error: {recommended.get('details', 'Unknown error')}")

⭐ Recommended Candidates:
{'viewed': True, 'text': '面议\n王斐\n刚刚活跃\n40岁10年以上硕士在职-月内到岗\n期望：\n北京自然语言处理算法\n优势：\n1.熟悉linux C++、python、java 2.熟悉transformer原理，Erine、BERT等预训练模型，以及常见的大模型原理GPT、Qwen、deepseek。4.了解RAG,Agent原理，熟悉dify，ragflow。5.熟悉常见NLP任务，如文本分类、命名实体识别、文本匹配等，以及Word2Vec、TF-IDF等文本向量化技术 6.熟悉mysql、redis、starrocks、elasticsearch等中间件，了解k8s，docker 7.具有5年的算法开发经验，多年的后端服务架构设计研发经验和领域驱动DDD经验 8.具有一定的团队管理经验及ToB行业经验 同时拥有多年的后端架构研发经验，和自然语言处理/深度学习经验，同时有一定的产品化思维，热爱Ai agent相关工作\n来自相似职位自然语言处理算法\n最近关注大模型算法\n量化技术\n预训练模型\n领域驱动\n2020.072023.07\n数势云创技术总监\n2009.072020.07\n百度在线网络技术（北京）自然语言处理算法\n20062009\n南开大学电子硕士\n20022006\n南开大学电子系本科\n打招呼'}
----------------------------------------------------------------------------------------------------
{'viewed': False, 'text': '35-45K\n潘军\n刚刚活跃\n35岁8年硕士在职-考虑机会\n期望：\n北京算法工程师\n优势：\n1.熟练掌握C++编程，熟悉面向对象的程序设计，算法设计和数据结构基础扎实。2.熟练运用Java、Python编程，熟悉springboot等框架。3.有OCR相关实践经历和技术积累，熟练掌握图像处理基本技术。4.了解Deep Learning 主要技术，熟悉pytorh框架。5.CET-6 501分，有良好英语听说读写能力。\n来自相似职位算法工程师\n视觉图像算法\n编程\n训练模型\n识别图

### View Recommended Candidate Resume


In [ ]:
# View resume of a recommended candidate (replace index with actual index)
candidate_index = 0  # Replace with actual index from recommended candidates

if 'recommended' in locals() and recommended.get('success'):
    view_result = requests.get(f'{BASE_URL}/recommend/candidate/{candidate_index}').json()
    
    print(f"👤 Recommended Candidate #{candidate_index} Resume:")
    print(json.dumps(view_result, indent=2, ensure_ascii=False))
else:
    print("⚠️ Please get recommended candidates first.")


👤 Recommended Candidate #0 Resume:
{
  "success": true,
  "text": "1.熟悉linux C++、python、java\n\n2.熟悉transformer原理，Erine、BERT等预训练模型，以及常见的大模型原理GPT、Qwen、deepseek。\n\n4.了解RAG, Agent原理，熟悉dify，ragflow。\n\n5.熟悉常见NLP任务，如文本分类、命名实体识别、文本匹配等，以及Word2Vec、TF-IDF等文本向量化技术\n\n6.熟悉mysql、redis、starrocks、elasticsearch等中间件，了解k8s，docker\n\n7.具有5年的算法开发经验，多年的后端服务架构设计研发经验和领域驱动DDD经验\n\n8.具有一定的团队管理经验及ToB行业经验\n\n\n\n同时拥有多年的后端架构研发经验，和自然语言处理/深度学习经验，同时有一定的产品化思维，热爱Ai agent相关工作\n期望职位\n北京自然语言处理算法行业不限面议\n工作经历\n北京数势云创科技有限公司\n技术总监\n2020.07 - 2023.07\n1. 自动化营销产品技术设计及实现\n\n2.自动化部署平台设计及实现\n!\n百度在线网络技术（北京）有限公司\n自然语言处理算法\n2009.07 - 2020.07\n1. UNIT智能对话平台-任务型对话机器人\n\n2. UNIT智能对话平台-FAQ问答型机器人\n\n3. 直达号网页质量评分系统\n\n4 直达号检索离线系统\n教育经历\n南开大学\n电子\n硕士\n2006 - 2009\n985院校\nQS世界大学排名TOP500\n南开大学\n电子系\n本科\n2002 - 2006\n985院校\nQS世界大学排名TOP500\n为妥善保护牛人在BOSS直聘平台提交、发布、展示的简历（包括但不限于在线简历、附件简历）中的个人信息（包括但不限于联系方式、期望职位、教育经历、工作经历等），任何用户原则上仅可出于自身招聘的目的，通过BOSS直聘平台在线浏览牛人简历。未经BOSS直聘及牛人本人书面授权，任何用户不得将牛人在BOSS直聘平台提交、发布、展示的简历中的个人信息，在任何第三方平台进行复制、使用、传播、存储。\n收藏\n不合适\

## Next Steps

1. **Customize the workflow** for your specific needs
2. **Set up automated scheduling** using cron jobs or task schedulers
3. **Integrate with your HR system** using the API endpoints
4. **Monitor and log** all automation activities
5. **Scale up** by running multiple instances for different job searches

## Support

For issues and questions:
- Check the service logs in `service.log`
- Use the debug endpoints to diagnose problems
- Ensure Chrome is running with debugging enabled
- Verify your Boss Zhipin account is properly logged in


In [ ]:
import base64
import os
from datetime import datetime

def save_resume_data(resume_result, chat_id, save_dir="resume_data"):
    """Save resume data to files"""
    if not resume_result.get('success'):
        print("❌ No data to save")
        return
    
    os.makedirs(save_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    saved_files = []
    
    # Save text content
    if resume_result.get('text'):
        text_file = os.path.join(save_dir, f"{chat_id}_text_{timestamp}.txt")
        with open(text_file, 'w', encoding='utf-8') as f:
            f.write(resume_result['text'])
        saved_files.append(text_file)
        print(f"✅ Text saved to: {text_file}")
    
    # Save main image
    if resume_result.get('image_base64'):
        image_file = os.path.join(save_dir, f"{chat_id}_main_{timestamp}.png")
        image_data = base64.b64decode(resume_result['image_base64'])
        with open(image_file, 'wb') as f:
            f.write(image_data)
        saved_files.append(image_file)
        print(f"✅ Main image saved to: {image_file}")
    
    # Save slice images
    if resume_result.get('images_base64'):
        for i, img_data in enumerate(resume_result['images_base64']):
            slice_file = os.path.join(save_dir, f"{chat_id}_slice_{i+1}_{timestamp}.png")
            image_data = base64.b64decode(img_data)
            with open(slice_file, 'wb') as f:
                f.write(image_data)
            saved_files.append(slice_file)
        print(f"✅ {len(resume_result['images_base64'])} slice images saved")
    
    return saved_files

# Save resume data if we have a valid result
if chat_id != "your_chat_id_here" and 'resume_result' in locals():
    saved_files = save_resume_data(resume_result, chat_id)
    if saved_files:
        print(f"\n📁 Total files saved: {len(saved_files)}")
else:
    print("⚠️ Please extract resume data first.")


# embedding


In [2]:
from src.assistant_actions import assistant_actions

text_sample = "候选人具有丰富的大模型训练经验，参与过多模态项目。"
embedding = assistant_actions.get_embedding(text_sample)
print(f"Embedding length: {len(embedding) if embedding else 'None'}")
print(embedding[:10] if embedding else "Embedding unavailable")


Embedding length: 1536
[0.01841905526816845, 0.0027341749519109726, 0.02852301113307476, -0.015655580908060074, 0.005107494071125984, 0.0178392194211483, 0.010788656771183014, 0.05240733176469803, -0.014458896592259407, 0.021762367337942123]


### Select Current Job

Select a specific job from the dropdown menu in the recommendation system.


In [29]:
# Select current job from dropdown menu
job_title = "大模型"  # Replace with your desired job title

response = requests.post(f"{BASE_URL}/recommend/select-job", 
                        json={"job_title": job_title}, 
                        timeout=30)

job_result = response.json()
print("🎯 Job Selection Result:")
print(json.dumps(job_result, indent=2, ensure_ascii=False))

if job_result.get('success'):
    print(f"✅ Successfully selected job: {job_result.get('selected_job')}")
else:
    print(f"❌ Failed to select job: {job_result.get('details')}")
    if job_result.get('available_jobs'):
        print("📋 Available jobs:")
        for job in job_result['available_jobs']:
            print(f"  - {job}")


🎯 Job Selection Result:
{
  "success": true,
  "details": "成功选择职位: 大模型算法工程师 _ 北京  25-40K",
  "selected_job": "大模型算法工程师 _ 北京 25-40K",
  "available_jobs": [
    "资深java开发工程师/架构师 _ 北京  40-70K",
    "大模型算法工程师 _ 北京  25-40K"
  ],
  "timestamp": "2025-09-29T21:20:10.485171"
}
✅ Successfully selected job: 大模型算法工程师 _ 北京 25-40K
